In [ ]:
from pathlib import Path
import shutil
import zipfile

# Kaggle の入力データセット slug（username/slug の slug 部分）をセット
DATASET_SLUG = 'gns-codes'  # 変更する場合はここを更新
DATASET_ROOT = Path(f'/kaggle/input/{DATASET_SLUG}')
WORK_ROOT = Path('/kaggle/working')
repo_dir = WORK_ROOT / 'code'

code_dir = DATASET_ROOT / 'code'
code_zip = DATASET_ROOT / 'code.zip'

# 取得優先順位: 展開済み code/ があればそれを使う。無ければ code.zip を展開。どちらも無ければエラー。
if code_dir.exists():
    src = code_dir
elif code_zip.exists():
    if repo_dir.exists():
        shutil.rmtree(repo_dir)
    with zipfile.ZipFile(code_zip) as zf:
        zf.extractall(repo_dir)
    src = repo_dir
else:
    raise FileNotFoundError(f"/kaggle/input/{DATASET_SLUG} に code ディレクトリも code.zip も見つかりません。Add Data で {DATASET_SLUG} を追加してください。")

if src != repo_dir:
    if repo_dir.exists():
        shutil.rmtree(repo_dir)
    shutil.copytree(src, repo_dir)
# unzip でサブディレクトリ1階層挟まった場合に平坦化
if repo_dir.exists():
    children = list(repo_dir.iterdir())
    if len(children)==1 and children[0].is_dir():
        inner = children[0]
        for path in inner.iterdir():
            path.rename(repo_dir / path.name)
        inner.rmdir()
%cd $repo_dir
# 依存パッケージは Kaggle 環境の標準ライブラリを利用（追加インストールなし）


In [ ]:
!pip install torch-geometric 
# torch-scatter torch-sparse torch-cluster


In [ ]:
import torch

torch_ver = torch.__version__.split("+")[0]          # 例: "2.1.0+cu121" -> "2.1.0"
cuda_ver  = torch.version.cuda                      # 例: "12.1" / CPUなら None
cuda_tag  = "cpu" if cuda_ver is None else "cu" + cuda_ver.replace(".", "")

url = f"https://data.pyg.org/whl/torch-{torch_ver}+{cuda_tag}.html"
print("PyG wheels:", url)

# 必要なやつだけ入れる（radius_graphなら torch_cluster が本体）
!pip -q install torch-cluster -f {url}

# もし他も使うならまとめて
# !pip -q install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f {url}
# !pip -q install torch_geometric


In [ ]:
from copy import deepcopy
from pathlib import Path
import re
import yaml

REPO = Path('/kaggle/working/code')
cfg_path = REPO / 'config_rollout.yaml'

# 推論で使うデータセット
dataset_root = '/kaggle/input/dam-break-left'

# 試したい dt を列挙
DT_LIST = [0.006, 0.003]  # ここを編集

# 出力先（階層を小さくするため rollouts 直下にまとめる）
output_root = REPO / 'rollouts'

# 可視化フォーマット: html|mp4|gif
VIZ_FORMAT = 'html'

# 追加でモデルを置いた Kaggle Dataset
# 例: /kaggle/input/gns-40k-100-20/pytorch/default/1
MODEL_DATASET_SLUG = 'gns-40k-100-20'
MODEL_SUBDIR = 'pytorch/default/1'
# モデルファイル名 (model-*.pt を自動探索するなら latest のまま)
MODEL_FILE = 'latest'

with cfg_path.open('r', encoding='utf-8') as f:
    base_cfg = yaml.safe_load(f)

base_cfg['method'] = 'gns'
base_cfg['rollout_inference_max_examples'] = 1
base_cfg.setdefault('scenario_options', {}).setdefault('fluid', {})['dataset'] = dataset_root

# used_config.json を書くので model_path は書き込み可能な場所に固定
base_cfg['model_path'] = str(REPO / 'models')

# 既にモデルが置かれている場所があれば自動で使う
model_candidates = []
if MODEL_DATASET_SLUG:
    dataset_root_path = Path(f'/kaggle/input/{MODEL_DATASET_SLUG}')
    model_candidates.append(dataset_root_path / MODEL_SUBDIR if MODEL_SUBDIR else dataset_root_path)
model_candidates.extend([
    Path('/kaggle/input/gns-codes') / 'models',
])

selected_model = None
resolved_model_file = None
for candidate in model_candidates:
    if not candidate.exists():
        continue
    if candidate.is_file():
        resolved_model_file = candidate
    else:
        if MODEL_FILE == 'latest':
            expr = re.compile(r"model-(\d+)\.pt$")
            best = None
            for path in candidate.rglob('model-*.pt'):
                m = expr.search(path.name)
                step = int(m.group(1)) if m else -1
                stat = path.stat()
                entry = (stat.st_mtime, step, path)
                if best is None or entry > best:
                    best = entry
            if best:
                resolved_model_file = best[2]
        else:
            resolved_model_file = candidate / MODEL_FILE
    selected_model = candidate
    if resolved_model_file is not None:
        break

if resolved_model_file is not None:
    base_cfg['model_file'] = str(resolved_model_file)

print(f"model candidates: {[str(p) for p in model_candidates]}")
print(f"selected model path: {selected_model}")
print(f"resolved model file: {resolved_model_file}")


def _dt_label(value: float) -> str:
    return str(value).replace('.', 'p')

cfg_variants = []
for dt in DT_LIST:
    cfg_dt = deepcopy(base_cfg)
    method_opts = cfg_dt.setdefault('method_options', {}).setdefault('hamiltonian_sph', {})
    integrator = method_opts.setdefault('integrator', {})
    integrator['dt'] = float(dt)
    integrator['dt_source'] = 'config'
    label = _dt_label(float(dt))
    # 出力階層を小さくするため output_path を固定し、ファイル名で区別
    cfg_dt['output_path'] = str(output_root)
    cfg_dt['output_filename'] = f"dt_{label}"
    cfg_variants.append((label, cfg_dt))


In [ ]:
%cd /kaggle/working/code

import subprocess
from pathlib import Path
import yaml

cfg_path = Path('config_rollout.yaml')
for label, cfg_dt in cfg_variants:
    with cfg_path.open('w', encoding='utf-8') as f:
        yaml.safe_dump(cfg_dt, f, allow_unicode=True)
    cmd = ["python", "src/train.py", "--config", "config_rollout.yaml"]
    print(f"[dt={label}] {' '.join(cmd)}")
    subprocess.run(cmd, check=True)

    # 出力先の pkl を可視化
    output_dir = Path(cfg_dt['output_path']) / cfg_dt['method'] / cfg_dt['output_filename']
    pkl_path = output_dir / f"{cfg_dt['output_filename']}_ex0.pkl"
    viz_out = output_dir / f"{cfg_dt['output_filename']}.{VIZ_FORMAT}"
    viz_cmd = [
        "python",
        "visualize_rollout.py",
        str(pkl_path),
        "--format",
        VIZ_FORMAT,
        "--output",
        str(viz_out),
    ]
    print(f"[dt={label}] {' '.join(viz_cmd)}")
    subprocess.run(viz_cmd, check=True)


In [ ]:
!ls -la /kaggle/input/gns-40k-100-20
